In [ ]:
# Pipeline from downloaded files, to complete huggingface dataset with metadata
# Normalisation of sound to [-1, 1]
# Construction of full dataset with metadata
# Train/Test/Val splits

In [ ]:
import os
import shutil

# Select chosen classes and copy across into data folder
class DatasetCreator:    
    _classes = []
    _dataset_path = "" # Where dataset will be constructed
    _class_path = "" # Where raw data is saved
    _preprocessor = None
    
    def __init__(self, dataset_path, class_path, preprocessor, *classes):
        self._dataset_path = dataset_path
        self._class_path = class_path
        self._classes = [c for c in classes]
        self._preprocessor = preprocessor
    
    # Creates corresponding folders for chosen classes
    def folder_setup(self):
        for folder in os.listdir(self._class_path):
            if folder in self._classes and os.path.isdir(os.path.join(self._class_path, folder)):
                dest_folder = os.path.join(self._dataset_path, folder)

                if not os.path.exists(dest_folder):
                    os.makedirs(dest_folder)

    # Copies files excluding those tagged with REMOVE
    def copy_files(self):
        for folder in os.listdir(self._class_path):  
            if os.path.isdir(os.path.join(self._class_path, folder)):
                for file in os.listdir(os.path.join(self._class_path, folder)):
                    if 'REMOVE' in file:
                        continue
                
                    src_file = os.path.join(self._class_path, folder, file)
                    dest_file = os.path.join(self._dataset_path, folder, file)
                    
                    shutil.copy(src_file, dest_file)
    
    # Converts to spectrogram
   
    
    def generate_metadata(self):
        

    def combine_data(self):
        
        
    def split_data(self, train_split, test_split, var_split):
        
        
    def create_dataset(self, train_split, test_split, var_split):

In [ ]:
import os
import librosa
import soundfile as sf
import numpy as np

class WavPreprocessor:
    def resample(self, audio, target_sr):
        y, sr = librosa.load(audio, sr=None)
        
        y_resampled = librosa.resample(y, sr, target_sr)
        
        return y_resampled, target_sr
    
    def resample_folder(self, input_path, output_path, target_sr):
        for filename in os.listdir(input_path):
            if filename.endswith('.wav'):
                path = os.path.join(input_path, filename)
                y_resampled, sr = self.resample(path, target_sr)
                
                save_path = os.path.join(output_path, 'resampled_' + str(sr) + '_' + filename)
                sf.write(save_path, y_resampled, sr)
    
    def min_max_normalise(self, audio):
        y, sr = librosa.load(audio, sr=None)
        
        normalised_y = y / np.max(np.abs(y))
        
        return normalised_y, sr
      
    def min_max_normalise_folder(self, input_path, output_path):
        for filename in os.listdir(input_path):
            if filename.endswith('.wav'):
                path = os.path.join(input_path, filename)
                norm_wav, sr = self.min_max_normalise(path)
                
                save_path = os.path.join(output_path, 'norm_' + filename)
                sf.write(save_path, norm_wav, sr)
                
    def wav_to_spec(self, wav):
        
    def spec_to_wav(self, spec): 
        
    def wav_to_spec_folder(self, input_path, output_path):
             
    def spec_to_wav_folder(self, input_path, output_path): 